In [2]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [3]:
# 加载数据
iris = load_iris()
data = iris.data
target = iris.target

In [10]:
# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2)
print("Train data length:", len(X_train))
print("Test data length:", len(X_test))

Train data length: 120
Test data length: 30


In [11]:
# 转换为Dataset数据格式
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [13]:
# 参数
params = {
    'task': 'train',
    'boosting_type': 'gbdt',  # 设置提升类型
    'objective': 'regression',  # 目标函数
    'metric': {'l2', 'auc'},  # 评估函数
    'num_leaves': 31,  # 叶子节点数
    'learning_rate': 0.05,  # 学习速率
    'feature_fraction': 0.9,  # 建树的特征选择比例
    'bagging_fraction': 0.8,  # 建树的样本采样比例
    'bagging_freq': 5,  # k 意味着每 k 次迭代执行bagging
    'verbose': 1  # <0 显示致命的, =0 显示错误 (警告), >0 显示信息
}

In [19]:
# 模型训练
gbm = lgb.train(params, lgb_train, num_boost_round=20, valid_sets=lgb_eval, early_stopping_rounds=10)

[1]	valid_0's l2: 0.609674	valid_0's auc: 1
Training until validation scores don't improve for 10 rounds
[2]	valid_0's l2: 0.558146	valid_0's auc: 1
[3]	valid_0's l2: 0.511555	valid_0's auc: 1
[4]	valid_0's l2: 0.469423	valid_0's auc: 1
[5]	valid_0's l2: 0.431795	valid_0's auc: 1
[6]	valid_0's l2: 0.395904	valid_0's auc: 1
[7]	valid_0's l2: 0.363404	valid_0's auc: 1
[8]	valid_0's l2: 0.334111	valid_0's auc: 1
[9]	valid_0's l2: 0.307527	valid_0's auc: 1
[10]	valid_0's l2: 0.283349	valid_0's auc: 1
[11]	valid_0's l2: 0.262388	valid_0's auc: 1
Early stopping, best iteration is:
[1]	valid_0's l2: 0.609674	valid_0's auc: 1


In [15]:
# 模型保存
gbm.save_model('model.txt')

In [16]:
# 模型加载
gbm = lgb.Booster(model_file='model.txt')

In [20]:
# 模型预测
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

In [21]:
# 模型评估
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

The rmse of prediction is: 0.7808163713020473
